## Working Instructions
#### Please change the paths as required to locate the files on respective systems. Thereafter, please run the complete file as a script. Run all cells.

In [26]:
# Importing the required files
import pandas as pd     # to read the input excel sheet
from bs4 import BeautifulSoup    # for scraping data from the website and navigating their structure
import requests  # to fetch the html content.
import os       # importing Operating System Interface i.e. os library
import string    # to remove punctuations
from nltk.tokenize import word_tokenize,sent_tokenize    # for tokenization
import re        # for pattern search

In [27]:
# reading the excel sheet with URLs
df=pd.read_excel(r'C:\Users\Admin\Desktop\Test Assignment\20211030 Test Assignment\Input.xlsx')
name=df.URL_ID  # defining name to the url_id column
URL=df['URL'].str.strip() # striping the trailing space in the URL column as it is causing issue

In [28]:
# Creating  a for loop here which can navigate through all the URLs in the sheet to extract the data.
for i, j in zip(URL,name):    # Zipping the columns 
    try:
        soup=BeautifulSoup(requests.get(i).text, 'lxml')    # requesting the html content from site and parsing it via 'lxml'
        Title=soup.find('title').text                       # extracting the title
        article=soup.find(attrs={'class': 'td-post-content'}).text  # extracting the article content
        result=Title+article                                        # concatinating title and article in one variable
        para=result.replace('\n', '').replace('\xa0','')            # replacing the non required expressions from the article
    
        with open('%s.txt'%j,'w', encoding='utf-8') as file:        # opening a text file to write the content extracted in para
            file.write(para)
            file.close                                              # closing the file.
    except Exception as e:                                          # tracking the error, if any occurs during the execution
        print('Error message:', i)
        print(e)
    

Error message: https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
'NoneType' object has no attribute 'text'
Error message: https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/
'NoneType' object has no attribute 'text'


Output Result:


Row 37 - blackassign0036 - 'Ooops... Error 404 \' 'Sorry, but the page you are looking for doesn't exist.'
https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/

Row 50 - blackassign0049 - 'Ooops... Error 404 \' 'Sorry, but the page you are looking for doesn't exist.'
https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/


## Stop Words File 

In [29]:
# storing and formatting the stopwords in one variable from the notepad
import os   # importing Operating System Interface i.e. os library

# Defining a function to collect stop words from different files in the file path by navigating through all the text file
def stop_file(file_path1):            
    file_names=os.listdir(file_path1)     # getting the list of files in the variable
    compiled_lines=[]                    # adding all the final words in this variable
    
    # looping for all the files
    for file_name in file_names:
        file_path_i=os.path.join(file_path1, file_name)  # full path of the individual files
        
        # Opening the file to read
        with open(file_path_i,'r',encoding='latin-1') as file:
            lines=file.readlines()     # reading the content of the file 
            lines=[line.strip() for line in lines]  # strip newline characters from each line
            compiled_lines.extend(lines)      #adding the line to compiled lines
    return compiled_lines
    

In [30]:
# Running the above defined function by providing the path of the directory where files are saved
compiled_lines=stop_file(r'C:\Users\Admin\Desktop\Test Assignment\20211030 Test Assignment\StopWords')

# replacing some hyperlinks in the data
compiled_lines[compiled_lines.index('JAMES  | http://www.census.gov/genealogy/names/dist.male.first')]='James'
compiled_lines[compiled_lines.index('LI  |  http://en.wikipedia.org/wiki/List_of_most_common_surnames#China')] = 'LI'

#replacing some characters from the compiled file and setting all the words in lower case with the help of list comprehension
modified_lines=[i.replace('|',',').replace('(', '').replace(')', '').replace('.', '') for i in compiled_lines]
stopwords_modified=[i.lower() for i in modified_lines]


## Data Cleaning & Tokenization

In [31]:
# defining function to clean the data, remove stop_words, tokenizing
def data_clean(file_path):
    file_names= os.listdir(file_path)           # getting the list of files in the variable
    cleaned_files=[]                             # initializing the variable to store namees of all the clean files
    final_words=[]                              # initializing the variable to add the final words

    
    for file_name in file_names:               # iterating through the file names directory
        file_name_individual=os.path.join(file_path, file_name)      # full path of the individual file
        cleanfile_name=os.path.splitext(file_name)[0]+'_clean.txt'   # setting the names of the clean files by using split extension
        
        if file_name.endswith('.txt'):                 # making sure that it only picks text files
            with open(file_name_individual,'r',encoding='utf-8') as text:            # opening individual files to read for text preprocessing
                file=text.read().lower()                                             
                tokenized_sent=(sent_tokenize(file))            # tokenizing the sentences
                num_sent=len(tokenized_sent)                    # counting the number of sentences
               
                
                clean_data=file.translate(str.maketrans('', '', string.punctuation)) 
                tokenized_words=(word_tokenize(clean_data))
                final_words=[word for word in tokenized_words if word not in stopwords_modified]
                count_of_words=len(final_words)                      # counting the number of words
                
            cleanfile_path=os.path.join(output_path, cleanfile_name)      # full path of the individual clean files
            with open(cleanfile_path, 'w', encoding='utf-8') as output_file:  # opening individual files to write
                for item in final_words:
                    output_file.write(f'"{item}",' )  # writing all the words in clean file
            
            cleaned_files.append((cleanfile_name, num_sent, count_of_words))    # appending the names of the clean file
                
    return cleaned_files    # output will be the list of cleaned files


In [32]:
file_path = r'C:\Users\Admin\BlackcOffer- Test Assignment'    # file path for text files for text preprocessing
output_path = r'C:\Users\Admin\BlackcOffer- Test Assignment\Clean'   # path for cleaned_files

In [33]:
# applying the function for data clean
cleaned_files= data_clean(file_path)   
print(cleaned_files)

[('blackassign0001_clean.txt', 57, 489), ('blackassign0002_clean.txt', 64, 708), ('blackassign0003_clean.txt', 46, 613), ('blackassign0004_clean.txt', 33, 592), ('blackassign0005_clean.txt', 26, 343), ('blackassign0006_clean.txt', 59, 1026), ('blackassign0007_clean.txt', 51, 652), ('blackassign0008_clean.txt', 36, 475), ('blackassign0009_clean.txt', 39, 581), ('blackassign0010_clean.txt', 108, 1639), ('blackassign0011_clean.txt', 44, 779), ('blackassign0012_clean.txt', 56, 824), ('blackassign0013_clean.txt', 19, 290), ('blackassign0014_clean.txt', 62, 521), ('blackassign0015_clean.txt', 44, 640), ('blackassign0016_clean.txt', 44, 640), ('blackassign0017_clean.txt', 43, 577), ('blackassign0018_clean.txt', 33, 562), ('blackassign0019_clean.txt', 73, 828), ('blackassign0020_clean.txt', 22, 227), ('blackassign0021_clean.txt', 41, 520), ('blackassign0022_clean.txt', 23, 176), ('blackassign0023_clean.txt', 54, 585), ('blackassign0024_clean.txt', 14, 277), ('blackassign0025_clean.txt', 18, 45

# Text Analysis

In [34]:
def text_analysis(location,path, path1):
    final_files=os.listdir(location)     # directory for files for text analysis
    result=[]     # creating variable to store the results
    
    
    with open(path,'r', encoding='latin-1') as positive_file,open(path1, 'r', encoding='latin-1') as negative_file:
        positive_words=set(positive_file.read().split())    # splitting the words 
        negative_words=set(negative_file.read().split())
       
        
        for final_file in final_files:
            final_file_i=os.path.join(location,final_file)    # full path for individual files
            
               
             
            with open(final_file_i,'r', encoding='latin-1') as p:     # opening and reading the individual files
                content = p.read()
                words = content.replace('"', '').split(',')      # as the file contain quotes and commas, replacing quotes and splitting by ','
                positive_score=0                                 # initializing scores
                negative_score=0
                for word in words:
                    if word in positive_words:                  # applying conditions
                        positive_score +=1
                            
                    elif word in negative_words:
                        negative_score +=1
                                              
                          
                result.append((positive_score, negative_score, final_file))     # appending results to final lists
            
    return result      

In [35]:
# providing paths for files
path= r'C:\Users\Admin\Desktop\Test Assignment\20211030 Test Assignment\MasterDictionary\positive-words.txt'
path1 = r'C:\Users\Admin\Desktop\Test Assignment\20211030 Test Assignment\MasterDictionary\negative-words.txt'
location=r'C:\Users\Admin\BlackcOffer- Test Assignment\Clean'

In [36]:
pos_neg_score=text_analysis(location,path, path1)   # retrieving the output

#### Polarity, Subjectivity Score

In [37]:
# Defining the function for polarity score
def polarity_score(positive_score, negative_score, file_name):
    polarity_score=[]                             #initializing the list to collect all scores
    for positive_score, negative_score, file_name in zip(pos_score, neg_score, file_names):    # looping through all the scores and files
        pscore=round((positive_score-negative_score)/((positive_score+negative_score)+ 0.000001),3)
        polarity_score.append((pscore,file_name))           # appending to the list
    return polarity_score




In [38]:
pos_score=[positive_score for positive_score,_,_ in pos_neg_score]   # defining positive and negative score list
neg_score=[negative_score for _,negative_score,_ in pos_neg_score]
file_names=[file_name for _,_, file_name in pos_neg_score]                # file names list

polarity_score=polarity_score(pos_score, neg_score, file_names)     # calling the function

In [39]:
# Defining function for counting the number of characters in a file.
def count_characters(location):
        output=[]
        clean_file_names=os.listdir(location)
        for clean_file in clean_file_names:
            clean_file_i=os.path.join(location,clean_file)
            
            with open(clean_file_i,'r', encoding='latin-1') as p:
                words=[word.strip() for word in p.read()]
                total_characters=len(words)                             # counting the characters
                output.append((total_characters, clean_file))
        return output

In [40]:
# calling the function by giving location as argument which is already defined earlier
output=count_characters(location)

In [43]:
# Defining function for subjectivity score
def sub_score(pos_score, neg_score, file_names, num_of_words):
    s_scores = []
    
    # zipping the lists together for iterations
    for pos, neg, file_name, num_words in zip(pos_score, neg_score, file_names, num_of_words):  
        sub_score = round((pos + neg) / (num_words + 0.000001), 3)
        s_scores.append((sub_score, file_name))
    return s_scores

#unpacking each tuple to get the third value and creating a new list with extracted values
num_of_words = [count_of_words for _, _, count_of_words in cleaned_files]

# calling the function
subjectivity_score = sub_score(pos_score, neg_score, file_names, num_of_words)


### Analysis of Readability

#### Average Sentence Length

In [45]:
# Defining function for average sent length which is same as avg words per sentence.
def average_sent_len(words,count_sent, file):
    average_length=round(words/count_sent,3)
    return average_length, file

In [47]:
num_words=[count_of_words for _,_,count_of_words in cleaned_files]
sent=[num_sent for _,num_sent,_ in cleaned_files]
average_len=[average_sent_len(words,count_sent, file) for words,count_sent, file in zip(num_words, sent, file_names)] 
print(average_len)

[(8.579, 'blackassign0001_clean.txt'), (11.062, 'blackassign0002_clean.txt'), (13.326, 'blackassign0003_clean.txt'), (17.939, 'blackassign0004_clean.txt'), (13.192, 'blackassign0005_clean.txt'), (17.39, 'blackassign0006_clean.txt'), (12.784, 'blackassign0007_clean.txt'), (13.194, 'blackassign0008_clean.txt'), (14.897, 'blackassign0009_clean.txt'), (15.176, 'blackassign0010_clean.txt'), (17.705, 'blackassign0011_clean.txt'), (14.714, 'blackassign0012_clean.txt'), (15.263, 'blackassign0013_clean.txt'), (8.403, 'blackassign0014_clean.txt'), (14.545, 'blackassign0015_clean.txt'), (14.545, 'blackassign0016_clean.txt'), (13.419, 'blackassign0017_clean.txt'), (17.03, 'blackassign0018_clean.txt'), (11.342, 'blackassign0019_clean.txt'), (10.318, 'blackassign0020_clean.txt'), (12.683, 'blackassign0021_clean.txt'), (7.652, 'blackassign0022_clean.txt'), (10.833, 'blackassign0023_clean.txt'), (19.786, 'blackassign0024_clean.txt'), (25.389, 'blackassign0025_clean.txt'), (13.0, 'blackassign0026_clean

#### Complex Words

In [48]:
# function for calculating number of complex words per file
vowels = ['a', 'e', 'i', 'o', 'u']

def cal_complex_words(location):
    complex_words_count = []
   
    clean_file_names = os.listdir(location)
    for clean_file in clean_file_names:
        clean_file_i = os.path.join(location, clean_file)    
            
        with open(clean_file_i, 'r', encoding='latin-1') as p:
            text=p.read()
            content=text.replace('"','').split(',')
            words = [word for word in content]
            complex_words=0
            for word in words:  
                count = 0

                for letter in word:
                    if letter.lower() in vowels:
                        count += 1
                    if word.endswith(('es', 'ed')):
                        count -= 1
                    if count>2:
                        complex_words+=1
                        break
                    
            complex_words_count.append((complex_words, clean_file))
           
                        
    return complex_words_count


In [49]:
# calling function
num=cal_complex_words(location)

In [50]:
# unpacking to retrieve targeted value for calculation and creating new lists
complex_num=[complex_words for complex_words,_ in num ]
clean_files=[clean_file for _,clean_file in num]
num_words=[count_of_words for _,_,count_of_words in cleaned_files]

In [51]:
# calculating the percentage of complex words
def Complex_percent(comp_words, words, file):
    comp_word_per=round((comp_words/words)*100,3)
    return comp_word_per, file

In [55]:
# calling the function
comp_word_percent=[Complex_percent(comp_words, words, file) for comp_words, words, file in zip(complex_num, num_words, clean_files)]
print(comp_word_percent)

[(39.059, 'blackassign0001_clean.txt'), (48.023, 'blackassign0002_clean.txt'), (53.834, 'blackassign0003_clean.txt'), (55.743, 'blackassign0004_clean.txt'), (46.064, 'blackassign0005_clean.txt'), (51.462, 'blackassign0006_clean.txt'), (47.393, 'blackassign0007_clean.txt'), (54.526, 'blackassign0008_clean.txt'), (56.282, 'blackassign0009_clean.txt'), (51.19, 'blackassign0010_clean.txt'), (56.098, 'blackassign0011_clean.txt'), (60.194, 'blackassign0012_clean.txt'), (59.655, 'blackassign0013_clean.txt'), (46.257, 'blackassign0014_clean.txt'), (50.313, 'blackassign0015_clean.txt'), (50.313, 'blackassign0016_clean.txt'), (47.834, 'blackassign0017_clean.txt'), (52.669, 'blackassign0018_clean.txt'), (52.053, 'blackassign0019_clean.txt'), (40.529, 'blackassign0020_clean.txt'), (54.038, 'blackassign0021_clean.txt'), (45.455, 'blackassign0022_clean.txt'), (44.615, 'blackassign0023_clean.txt'), (50.542, 'blackassign0024_clean.txt'), (33.917, 'blackassign0025_clean.txt'), (42.094, 'blackassign0026

#### Fog Index

In [56]:
# function defining fog index
def fog_index(avg_sen_len, comp_w_per, file):
    fog_index=round(0.4 * (avg_sen_len + comp_w_per),3)
    return fog_index, file

In [57]:
sent_len=[average_length for average_length,_ in average_len]
complex_per=[percent for percent,_ in comp_word_percent]

# calling the function
f_index=[fog_index(avg_sen_len, comp_w_per, file) for avg_sen_len, comp_w_per, file in zip(sent_len, complex_per, clean_files) ]
print(f_index)

[(19.055, 'blackassign0001_clean.txt'), (23.634, 'blackassign0002_clean.txt'), (26.864, 'blackassign0003_clean.txt'), (29.473, 'blackassign0004_clean.txt'), (23.702, 'blackassign0005_clean.txt'), (27.541, 'blackassign0006_clean.txt'), (24.071, 'blackassign0007_clean.txt'), (27.088, 'blackassign0008_clean.txt'), (28.472, 'blackassign0009_clean.txt'), (26.546, 'blackassign0010_clean.txt'), (29.521, 'blackassign0011_clean.txt'), (29.963, 'blackassign0012_clean.txt'), (29.967, 'blackassign0013_clean.txt'), (21.864, 'blackassign0014_clean.txt'), (25.943, 'blackassign0015_clean.txt'), (25.943, 'blackassign0016_clean.txt'), (24.501, 'blackassign0017_clean.txt'), (27.88, 'blackassign0018_clean.txt'), (25.358, 'blackassign0019_clean.txt'), (20.339, 'blackassign0020_clean.txt'), (26.688, 'blackassign0021_clean.txt'), (21.243, 'blackassign0022_clean.txt'), (22.179, 'blackassign0023_clean.txt'), (28.131, 'blackassign0024_clean.txt'), (23.722, 'blackassign0025_clean.txt'), (22.038, 'blackassign0026

#### Average Word Length

In [58]:
# Defining function counting the average word length
def Avg_word_len(num_char, word_count, file):
    Avg_word_lens=[]
    for num_char, word_count, file in zip(numchar, wordcount, clean_files ):       
        Avg_word_len=num_char/word_count
        Avg_word_lens.append((Avg_word_len, file))
    return Avg_word_lens

In [60]:
# unpacking the values, creating lists and calling the function
wordcount=[count_of_words for _,_,count_of_words in cleaned_files]
numchar= [total_characters for total_characters,_ in output]
Avg_word=Avg_word_len(numchar, wordcount, clean_files)

#### Personal Pronouns

In [64]:
# Defining function to search personal pronouns in the text files using regex
def personal_pronouns(file_loc):
    file_names= os.listdir(file_loc)
    p_noun_count=[]
    
    for file_name in file_names:
        file_name_individual=os.path.join(file_loc, file_name)
        if file_name.endswith('.txt'):
            p_noun=0
            with open(file_name_individual,'r',encoding='utf-8') as text:
                file=text.read()  
                clean_data=file.translate(str.maketrans('', '', string.punctuation))
                tokenized_words=word_tokenize(clean_data)
                
                Personal_pronoun = re.compile(r'\b(I|we|We|my|My|ours|Ours|us)\b')

                for words in tokenized_words:
                    match=re.findall(Personal_pronoun, words)
                    p_noun+=len(match)
            p_noun_count.append((p_noun, file_name))
            
    return p_noun_count



In [65]:
# providing location of the files here separately as it is to be done on file before removing stop words
file_loc=r'C:\Users\Admin\BlackcOffer- Test Assignment'
pcount=personal_pronouns(file_loc)

#### Output Updation to Excel

In [66]:
output_file=r'C:\Users\Admin\Desktop\Test Assignment\20211030 Test Assignment\Output Data Structure.xlsx'
df=pd.read_excel(r'C:\Users\Admin\Desktop\Test Assignment\20211030 Test Assignment\Output Data Structure.xlsx')

for p_noun, file_name in pcount:
    index=df[df['URL_ID']==file_name.replace('.txt', '')].index      

    if not index.empty:
        df.at[index[0], 'PERSONAL PRONOUNS']=p_noun
        
df.to_excel(output_file, index=False)

In [68]:
for positive_score, negative_score, final_file in  pos_neg_score:
    index=df[df['URL_ID']==final_file.replace('_clean.txt', '')].index      

    if not index.empty:
        df.at[index[0], 'POSITIVE SCORE']=positive_score
        df.at[index[0], 'NEGATIVE SCORE']=negative_score
        
        
df.to_excel(output_file, index=False)

In [69]:
for pscore, file_name in polarity_score:
        index=df[df['URL_ID']==file_name.replace('_clean.txt', '')].index      
        
        if not index.empty:
            df.at[index[0], 'POLARITY SCORE']=pscore
        
df.to_excel(output_file, index=False)

In [70]:
for sub_score, file_name in subjectivity_score:
    index=df[df['URL_ID']==file_name.replace('_clean.txt', '')].index 

    if not index.empty:
        df.at[index[0], 'SUBJECTIVITY SCORE']=sub_score

        
df.to_excel(output_file, index=False)

In [71]:
# Same output as both are same. Hence updating both with same outputs

for average_length, file in average_len:
    index=df[df['URL_ID']==file.replace('_clean.txt', '')].index 

    if not index.empty:
        df.at[index[0], 'AVG SENTENCE LENGTH']=average_length
        df.at[index[0], 'AVG NUMBER OF WORDS PER SENTENCE']=average_length
        

        
df.to_excel(output_file, index=False)

In [72]:
for complex_words, clean_file in num:
    index=df[df['URL_ID']==clean_file.replace('_clean.txt', '')].index 

    if not index.empty:
        df.at[index[0], 'COMPLEX WORD COUNT']=complex_words

        
df.to_excel(output_file, index=False)

In [73]:
for comp_word_per, file in comp_word_percent:
    index=df[df['URL_ID']==file.replace('_clean.txt', '')].index 

    if not index.empty:
        df.at[index[0], 'PERCENTAGE OF COMPLEX WORDS']=comp_word_per

        
df.to_excel(output_file, index=False)

In [74]:
for fog_index, file in f_index:
    index=df[df['URL_ID']==file.replace('_clean.txt', '')].index 

    if not index.empty:
        df.at[index[0], 'FOG INDEX']=fog_index

        
df.to_excel(output_file, index=False)
